In [93]:
import networkx as nx
import numpy as np
import random

In [26]:
size_1 = 50
size_2 = 50

In [37]:
sizes = [size_1, size_2]
probs = [[0.01, 0.01],
         [0.01, 0.01]]

In [50]:
g = nx.generators.community.stochastic_block_model(sizes, probs, seed=11)

In [51]:
g.edges()

EdgeView([(1, 3), (1, 59), (4, 51), (5, 78), (5, 85), (7, 86), (9, 95), (9, 97), (10, 97), (17, 69), (19, 60), (20, 47), (20, 53), (25, 86), (27, 93), (28, 80), (28, 95), (29, 98), (30, 47), (30, 61), (31, 88), (33, 81), (33, 98), (35, 94), (36, 69), (39, 72), (41, 78), (44, 97), (46, 57), (49, 87), (53, 54), (57, 80), (65, 98), (66, 68), (67, 78), (74, 99), (83, 93)])

In [52]:
g.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99))

In [62]:
bichromatic_ratio = []
for x in range(5000):
    g = nx.generators.community.stochastic_block_model(sizes, probs, seed=x)
    bichromatic = 0
    for (x,y) in g.edges():
        if x // size_1 != y // size_2:
            bichromatic += 1
    bichromatic_ratio.append(bichromatic / len(g.edges()))

In [25]:
26 // 25

1

In [63]:
np.average(bichromatic_ratio)

0.5041452851852587

In [65]:
np.max(bichromatic_ratio) - np.min(bichromatic_ratio)

0.54970375246872938

## Create graph

In [66]:
G = nx.Graph()
G.add_nodes_from(range(graph_size))

In [173]:
def get_bichromatic_ratio(g):
    bichromatic = 0
    for (x,y) in g.edges():
        if x // size_1 != y // size_2:
            bichromatic += 1
    return bichromatic / len(g.edges())

In [174]:
graph_size = 100
maj_min_ratio = .5
size_maj = int(maj_min_ratio * graph_size)
size_min = int(graph_size - size_1)
probs = [[0.05, 0.05],
         [0.05, 0.05]]
g = nx.generators.community.stochastic_block_model([size_maj, size_min], probs, seed=0)
bichromatic_ratios = [get_bichromatic_ratio(g)]

In [175]:
bichromatic_ratios

[0.449438202247191]

In [176]:
len(g.edges())

267

In [177]:
iterations = 20
new_friends_per_iter = 5

In [178]:
def get_group_size(node):
    if node < size_maj:
        return size_maj
    else:
        return size_min

In [179]:
for _ in range(iterations):
    # Select random node
    node = np.random.randint(0, graph_size)
    edges_added = 0
    # Add edges of friends of friends randomly
    while (edges_added < new_friends_per_iter):
        if len(set(nx.all_neighbors(g, node)).intersection(set(range(get_group_size(node))))) > 0:
            friend = random.sample(set(nx.all_neighbors(g, node)).intersection(set(range(get_group_size(node)))), 1)[0]
            new_possible_friends = set(nx.all_neighbors(g, friend)).difference(set(nx.all_neighbors(g, node)))
            if len(new_possible_friends) > 0:
                new_friend = random.sample(new_possible_friends, 1)[0]
                g.add_edge(node, new_friend)
                edges_added += 1
    bichromatic_ratios.append(get_bichromatic_ratio(g))

In [180]:
len(g.edges())

367

In [181]:
get_group_size(30)

50

In [182]:
bichromatic_ratios

[0.449438202247191,
 0.4485294117647059,
 0.4548736462093863,
 0.4574468085106383,
 0.4564459930313589,
 0.4554794520547945,
 0.45454545454545453,
 0.45695364238410596,
 0.4592833876221498,
 0.46153846153846156,
 0.4605678233438486,
 0.45962732919254656,
 0.45565749235474007,
 0.45180722891566266,
 0.4540059347181009,
 0.45321637426900585,
 0.45821325648414984,
 0.45738636363636365,
 0.4565826330532213,
 0.4558011049723757,
 0.45776566757493187]